In [1]:
#import IPython;IPython.embed()
from __future__ import division

import os
import sys
import time
import datetime
import argparse

# Import things from actual YOLO folder
sys.path.insert(0, '/root/PyTorch-YOLOv3')

from models import *
from utils.utils import *
from utils.datasets import *

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import matplotlib
#matplotlib.use('agg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import NullLocator

import cv2
from scipy.misc import imresize

# Need?
%matplotlib inline 

print("Imported all successfully!")

Imported all successfully!


#### Parameters to always use

In [2]:
config_path = "../PyTorch-YOLOv3/config/yolov3.cfg"
weights_path = "../PyTorch-YOLOv3/weights/yolov3.weights"
class_path = "../PyTorch-YOLOv3/data/coco.names"
conf_thres = 0.8
nms_thres = 0.4
batch_size = 1
n_cpu = 8
img_size = 416
use_cuda = True
cuda = torch.cuda.is_available() and use_cuda # Whether the gpu is available
model = Darknet(config_path, img_size=img_size) # Set up model
model.load_weights(weights_path)
if cuda:
    model.cuda()
model.eval() # Set in evaluation mode
classes = load_classes(class_path) # Extracts class labels from file

/root/anaconda3/lib/python3.6/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [3]:
def draw_frame(frame, detections):
    # make sure return is numpy array of same dimensions as input frame
    frame = imresize(frame, (720, 1280))
    
    if detections is None or detections[0] is None:
        return frame

    # The amount of padding that was added
    pad_x = max(frame.shape[0] - frame.shape[1], 0) * (img_size / max(frame.shape))
    pad_y = max(frame.shape[1] - frame.shape[0], 0) * (img_size / max(frame.shape))
    pad_y = 0
    # Image height and width after padding is removed
    unpad_h = img_size - pad_y
    unpad_w = img_size - pad_x
    
    # Draw bounding boxes and labels of detections
    unique_labels = detections[0][:, -1].cpu().unique()
    n_cls_preds = len(unique_labels)
    #bbox_colors = random.sample(colors, n_cls_preds)
    for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections[0]:    # data in detection array!
        # Rescale coordinates to original dimensions
        box_h = ((y2 - y1) / unpad_h) * frame.shape[0]
        box_w = ((x2 - x1) / unpad_w) * frame.shape[1]
        y1 = ((y1 - pad_y // 2) / unpad_h) * frame.shape[0]
        x1 = ((x1 - pad_x // 2) / unpad_w) * frame.shape[1]

        #color = bbox_colors[int(np.where(unique_labels == int(cls_pred))[0])]
        cv2.rectangle(frame, (x1, y1), (x1 + box_w, y1 + box_h), (255,128,0), 3)
        cv2.putText(frame, classes[int(cls_pred)], (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (255,255,255), 2)
        
        #import IPython;IPython.embed()
        
    return frame

In [4]:
def infer_draw_frame(frame):    
    # Convert the frame into a tensor
    frame_tensor = torch.from_numpy(frame)
    frame_tensor = frame_tensor.unsqueeze(0) # Add the batch dimension
    frame_tensor = frame_tensor.permute(0, 3, 1, 2) # Swap axes to [b,c,w,h] form
    frame_tensor = frame_tensor.float() # Convert byte tensor to float tensor
    frame_tensor = frame_tensor / 255   # Normalize data
        
    Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
    start_time = time.time()

    with torch.no_grad():
        detections = model(frame_tensor)
        detections = non_max_suppression(detections, 80, conf_thres, nms_thres)
                
    current_time = time.time()
    inference_time = datetime.timedelta(seconds=current_time - start_time)
    print ("\tInferrence time: ", inference_time)
        
    final_frame = draw_frame(frame, detections)
    return final_frame

### Main Loop

In [5]:
video_input_filename = "input-video.avi"
video_output_filename = "OUTPUT_VIDEO.avi"
# Create video capture object to read video frames
vid_read = cv2.VideoCapture(video_input_filename)
TOTAL_FRAMES = int(vid_read.get(cv2.CAP_PROP_FRAME_COUNT))
print("Number of frames: ", TOTAL_FRAMES)
# Create video writer object
fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
vid_write = cv2.VideoWriter(video_output_filename, fourcc, 30.0, (1280, 720))

Number of frames:  1260


In [6]:
success, frame = vid_read.read()
print("Data type of frame: ", type(frame))
print("Frame size: ", frame.shape, "\tin width height channels")
frame_num = 0
while success:
    print("Frame ", frame_num, "/", TOTAL_FRAMES, "=", "%0.2f" % (frame_num / TOTAL_FRAMES), end="")
    # Resize numpy array to correct size
    resized_frame = imresize(frame, (img_size, img_size))
    output_frame = infer_draw_frame(resized_frame)    
    vid_write.write(output_frame)
    # Read next frame from input
    success, frame = vid_read.read()
    frame_num += 1    
    
# Clean up    
vid_read.release()
vid_write.release()
print("Done!")

Data type of frame:  <class 'numpy.ndarray'>
Frame size:  (720, 1280, 3) 	in width height channels
Frame  0 / 1260 = 0.00

/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  
/root/anaconda3/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  This is separate from the ipykernel package so we can avoid doing imports until


	Inferrence time:  0:00:00.663627
Frame  1 / 1260 = 0.00	Inferrence time:  0:00:00.600036
Frame  2 / 1260 = 0.00	Inferrence time:  0:00:00.607792
Frame  3 / 1260 = 0.00	Inferrence time:  0:00:00.610832
Frame  4 / 1260 = 0.00	Inferrence time:  0:00:00.594254
Frame  5 / 1260 = 0.00	Inferrence time:  0:00:00.600249
Frame  6 / 1260 = 0.00	Inferrence time:  0:00:00.590404
Frame  7 / 1260 = 0.01	Inferrence time:  0:00:00.590880
Frame  8 / 1260 = 0.01	Inferrence time:  0:00:00.604321
Frame  9 / 1260 = 0.01	Inferrence time:  0:00:00.592949
Frame  10 / 1260 = 0.01	Inferrence time:  0:00:00.601610
Frame  11 / 1260 = 0.01	Inferrence time:  0:00:00.588794
Frame  12 / 1260 = 0.01	Inferrence time:  0:00:00.589694
Frame  13 / 1260 = 0.01	Inferrence time:  0:00:00.593164
Frame  14 / 1260 = 0.01	Inferrence time:  0:00:00.590595
Frame  15 / 1260 = 0.01	Inferrence time:  0:00:00.589399
Frame  16 / 1260 = 0.01	Inferrence time:  0:00:00.589818
Frame  17 / 1260 = 0.01	Inferrence time:  0:00:00.593353
Frame 

Frame  144 / 1260 = 0.11	Inferrence time:  0:00:00.614537
Frame  145 / 1260 = 0.12	Inferrence time:  0:00:00.599057
Frame  146 / 1260 = 0.12	Inferrence time:  0:00:00.615049
Frame  147 / 1260 = 0.12	Inferrence time:  0:00:00.600722
Frame  148 / 1260 = 0.12	Inferrence time:  0:00:00.586845
Frame  149 / 1260 = 0.12	Inferrence time:  0:00:00.587713
Frame  150 / 1260 = 0.12	Inferrence time:  0:00:00.589684
Frame  151 / 1260 = 0.12	Inferrence time:  0:00:00.595956
Frame  152 / 1260 = 0.12	Inferrence time:  0:00:00.587689
Frame  153 / 1260 = 0.12	Inferrence time:  0:00:00.592735
Frame  154 / 1260 = 0.12	Inferrence time:  0:00:00.585235
Frame  155 / 1260 = 0.12	Inferrence time:  0:00:00.589871
Frame  156 / 1260 = 0.12	Inferrence time:  0:00:00.586905
Frame  157 / 1260 = 0.12	Inferrence time:  0:00:00.582756
Frame  158 / 1260 = 0.13	Inferrence time:  0:00:00.585655
Frame  159 / 1260 = 0.13	Inferrence time:  0:00:00.587957
Frame  160 / 1260 = 0.13	Inferrence time:  0:00:00.592571
Frame  161 / 1

Frame  285 / 1260 = 0.23	Inferrence time:  0:00:00.587569
Frame  286 / 1260 = 0.23	Inferrence time:  0:00:00.592362
Frame  287 / 1260 = 0.23	Inferrence time:  0:00:00.592255
Frame  288 / 1260 = 0.23	Inferrence time:  0:00:00.587934
Frame  289 / 1260 = 0.23	Inferrence time:  0:00:00.588790
Frame  290 / 1260 = 0.23	Inferrence time:  0:00:00.590585
Frame  291 / 1260 = 0.23	Inferrence time:  0:00:00.591372
Frame  292 / 1260 = 0.23	Inferrence time:  0:00:00.597663
Frame  293 / 1260 = 0.23	Inferrence time:  0:00:00.588362
Frame  294 / 1260 = 0.23	Inferrence time:  0:00:00.587335
Frame  295 / 1260 = 0.23	Inferrence time:  0:00:00.590502
Frame  296 / 1260 = 0.23	Inferrence time:  0:00:00.586783
Frame  297 / 1260 = 0.24	Inferrence time:  0:00:00.591831
Frame  298 / 1260 = 0.24	Inferrence time:  0:00:00.587871
Frame  299 / 1260 = 0.24	Inferrence time:  0:00:00.587267
Frame  300 / 1260 = 0.24	Inferrence time:  0:00:00.590269
Frame  301 / 1260 = 0.24	Inferrence time:  0:00:00.590804
Frame  302 / 1

...

Frame  567 / 1260 = 0.45	Inferrence time:  0:00:00.586578
Frame  568 / 1260 = 0.45	Inferrence time:  0:00:00.594517
Frame  569 / 1260 = 0.45	Inferrence time:  0:00:00.590722
Frame  570 / 1260 = 0.45	Inferrence time:  0:00:00.586961
Frame  571 / 1260 = 0.45	Inferrence time:  0:00:00.590878
Frame  572 / 1260 = 0.45	Inferrence time:  0:00:00.586020
Frame  573 / 1260 = 0.45	Inferrence time:  0:00:00.590678
Frame  574 / 1260 = 0.46	Inferrence time:  0:00:00.588343
Frame  575 / 1260 = 0.46	Inferrence time:  0:00:00.590603
Frame  576 / 1260 = 0.46	Inferrence time:  0:00:00.589077
Frame  577 / 1260 = 0.46	Inferrence time:  0:00:00.586734
Frame  578 / 1260 = 0.46	Inferrence time:  0:00:00.588244
Frame  579 / 1260 = 0.46	Inferrence time:  0:00:00.593730
Frame  580 / 1260 = 0.46	Inferrence time:  0:00:00.587620
Frame  581 / 1260 = 0.46	Inferrence time:  0:00:00.589617
Frame  582 / 1260 = 0.46	Inferrence time:  0:00:00.593451
Frame  583 / 1260 = 0.46	Inferrence time:  0:00:00.588028
Frame  584 / 1

Frame  708 / 1260 = 0.56	Inferrence time:  0:00:00.594188
Frame  709 / 1260 = 0.56	Inferrence time:  0:00:00.589270
Frame  710 / 1260 = 0.56	Inferrence time:  0:00:00.587212
Frame  711 / 1260 = 0.56	Inferrence time:  0:00:00.587383
Frame  712 / 1260 = 0.57	Inferrence time:  0:00:00.592973
Frame  713 / 1260 = 0.57	Inferrence time:  0:00:00.588645
Frame  714 / 1260 = 0.57	Inferrence time:  0:00:00.588982
Frame  715 / 1260 = 0.57	Inferrence time:  0:00:00.592360
Frame  716 / 1260 = 0.57	Inferrence time:  0:00:00.589254
Frame  717 / 1260 = 0.57	Inferrence time:  0:00:00.591329
Frame  718 / 1260 = 0.57	Inferrence time:  0:00:00.592640
Frame  719 / 1260 = 0.57	Inferrence time:  0:00:00.588471
Frame  720 / 1260 = 0.57	Inferrence time:  0:00:00.586543
Frame  721 / 1260 = 0.57	Inferrence time:  0:00:00.591660
Frame  722 / 1260 = 0.57	Inferrence time:  0:00:00.589488
Frame  723 / 1260 = 0.57	Inferrence time:  0:00:00.586629
Frame  724 / 1260 = 0.57	Inferrence time:  0:00:00.588482
Frame  725 / 1

Frame  849 / 1260 = 0.67	Inferrence time:  0:00:00.589674
Frame  850 / 1260 = 0.67	Inferrence time:  0:00:00.589242
Frame  851 / 1260 = 0.68	Inferrence time:  0:00:00.590607
Frame  852 / 1260 = 0.68	Inferrence time:  0:00:00.587328
Frame  853 / 1260 = 0.68	Inferrence time:  0:00:00.585081
Frame  854 / 1260 = 0.68	Inferrence time:  0:00:00.590556
Frame  855 / 1260 = 0.68	Inferrence time:  0:00:00.591165
Frame  856 / 1260 = 0.68	Inferrence time:  0:00:00.590874
Frame  857 / 1260 = 0.68	Inferrence time:  0:00:00.591307
Frame  858 / 1260 = 0.68	Inferrence time:  0:00:00.588854
Frame  859 / 1260 = 0.68	Inferrence time:  0:00:00.589016
Frame  860 / 1260 = 0.68	Inferrence time:  0:00:00.589840
Frame  861 / 1260 = 0.68	Inferrence time:  0:00:00.591775
Frame  862 / 1260 = 0.68	Inferrence time:  0:00:00.587385
Frame  863 / 1260 = 0.68	Inferrence time:  0:00:00.591553
Frame  864 / 1260 = 0.69	Inferrence time:  0:00:00.590625
Frame  865 / 1260 = 0.69	Inferrence time:  0:00:00.589896
Frame  866 / 1

Frame  990 / 1260 = 0.79	Inferrence time:  0:00:00.592293
Frame  991 / 1260 = 0.79	Inferrence time:  0:00:00.592979
Frame  992 / 1260 = 0.79	Inferrence time:  0:00:00.589478
Frame  993 / 1260 = 0.79	Inferrence time:  0:00:00.586174
Frame  994 / 1260 = 0.79	Inferrence time:  0:00:00.602502
Frame  995 / 1260 = 0.79	Inferrence time:  0:00:00.590906
Frame  996 / 1260 = 0.79	Inferrence time:  0:00:00.591268
Frame  997 / 1260 = 0.79	Inferrence time:  0:00:00.587115
Frame  998 / 1260 = 0.79	Inferrence time:  0:00:00.590463
Frame  999 / 1260 = 0.79	Inferrence time:  0:00:00.588825
Frame  1000 / 1260 = 0.79	Inferrence time:  0:00:00.588242
Frame  1001 / 1260 = 0.79	Inferrence time:  0:00:00.588976
Frame  1002 / 1260 = 0.80	Inferrence time:  0:00:00.590488
Frame  1003 / 1260 = 0.80	Inferrence time:  0:00:00.588011
Frame  1004 / 1260 = 0.80	Inferrence time:  0:00:00.589161
Frame  1005 / 1260 = 0.80	Inferrence time:  0:00:00.589466
Frame  1006 / 1260 = 0.80	Inferrence time:  0:00:00.590933
Frame  

Frame  1129 / 1260 = 0.90	Inferrence time:  0:00:00.590356
Frame  1130 / 1260 = 0.90	Inferrence time:  0:00:00.587902
Frame  1131 / 1260 = 0.90	Inferrence time:  0:00:00.586623
Frame  1132 / 1260 = 0.90	Inferrence time:  0:00:00.589034
Frame  1133 / 1260 = 0.90	Inferrence time:  0:00:00.589914
Frame  1134 / 1260 = 0.90	Inferrence time:  0:00:00.585975
Frame  1135 / 1260 = 0.90	Inferrence time:  0:00:00.589952
Frame  1136 / 1260 = 0.90	Inferrence time:  0:00:00.587165
Frame  1137 / 1260 = 0.90	Inferrence time:  0:00:00.592492
Frame  1138 / 1260 = 0.90	Inferrence time:  0:00:00.590930
Frame  1139 / 1260 = 0.90	Inferrence time:  0:00:00.583864
Frame  1140 / 1260 = 0.90	Inferrence time:  0:00:00.594941
Frame  1141 / 1260 = 0.91	Inferrence time:  0:00:00.590180
Frame  1142 / 1260 = 0.91	Inferrence time:  0:00:00.590298
Frame  1143 / 1260 = 0.91	Inferrence time:  0:00:00.589626
Frame  1144 / 1260 = 0.91	Inferrence time:  0:00:00.588329
Frame  1145 / 1260 = 0.91	Inferrence time:  0:00:00.5863